In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

Below I have code to collect all the parquet files in a list and than concataning the files together and printing them in a wrong way. But because I didn't want to keep doing it. It is commented out. I than have the file being read into them.Than I wanted to check shape and columns.

In [2]:
# # Get a list of all Parquet files in the directory
# files = glob.glob(os.path.expanduser('~/Downloads/*.parquet'))
# print(files)  # printing file names to verify

# # Initialize an empty list to hold DataFrames
# df_list = []

# # Read each file into a DataFrame and append it to the list
# for file in files:
#     df_read = pd.read_parquet(file)
#     df_list.append(df_read)

# # Concatenate all DataFrames in the list
# if df_list:  # Checking if the list is not empty
#     df_rows = pd.concat(df_list, ignore_index=True)
#     # Write the combined DataFrame to a new Parquet file
#     df_rows.to_parquet('combined.parquet')
# else:
#     print("No parquet files found in the directory.")

In [3]:
df = pd.read_parquet('combined.parquet')
df.shape

(39656098, 19)

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

## Logical tests 
###  logical test 1  times being accurate or correct.
Ok I did a few logical tests to  check integrity of the data. So the first test is ensuring that the times for drop off are after pick up time and creating a column to indicate if its invalid trip durations. Ant than wanted to check when it is.I have this amount when I run it. So this is a big proportion that I think should be dropped. But analysing them all together to see if all logical tests are valid for all rows. [13613 rows x 20 columns]

In [5]:
# Check if the dropoff time is earlier than the pickup time for any record
invalid_records = df[df['tpep_dropoff_datetime'] < df['tpep_pickup_datetime']]

# Add a column 'invalid_time' to indicate invalid trip durations
df['invalid_time'] = df['tpep_dropoff_datetime'] < df['tpep_pickup_datetime']

# Print the records with invalid trip durations, if any
invalid_records = df[df['invalid_time']]

# Print the invalid records, if any
if not invalid_records.empty:
    print("Invalid trip duration detected in the following records:")
    print(invalid_records)
else:
    print("All trip durations are valid.")

Invalid trip duration detected in the following records:
          VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
1780087          1  2022-10-16 23:48:14   2022-10-16 00:30:42   
2418647          1  2022-10-22 10:00:00   2022-10-22 09:26:56   
3543244          6  2022-10-01 05:10:58   2022-10-01 05:10:24   
3543780          6  2022-10-01 10:10:16   2022-10-01 10:10:06   
3543799          6  2022-10-01 10:10:51   2022-10-01 10:10:37   
...            ...                  ...                   ...   
39655298         6  2022-05-31 17:05:26   2022-05-31 17:05:24   
39655461         6  2022-05-31 18:05:38   2022-05-31 18:05:00   
39655541         6  2022-05-31 18:05:17   2022-05-31 18:05:06   
39655599         6  2022-05-31 19:05:45   2022-05-31 19:05:10   
39656003         6  2022-05-31 22:05:31   2022-05-31 22:05:22   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
1780087               1.0          21.70         2.0                  N   
2418647     

### Logical tests 2 trip distance ensuring it is not 0 or below.
Next test is to see if the trip distance is not below 0 as it would not make sense that a negative taxi journey would happen.

In [6]:
# Check for Invalid Trip Distances
df['invalid_trip_distance'] = df['trip_distance'] <= 0

# Print the records with invalid trip distances, if any
invalid_trip_distances = df[df['invalid_trip_distance']]
if not invalid_trip_distances.empty:
    print("Invalid trip distances detected in the following records:")
    print(invalid_trip_distances)
else:
    print("All trip distances are valid.")

Invalid trip distances detected in the following records:
          VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
293              2  2022-10-01 00:46:38   2022-10-01 00:46:44   
294              2  2022-10-01 00:46:38   2022-10-01 00:46:44   
362              1  2022-10-01 00:32:03   2022-10-01 00:49:48   
431              1  2022-10-01 00:41:34   2022-10-01 00:44:51   
520              2  2022-10-01 00:59:21   2022-10-01 00:59:25   
...            ...                  ...                   ...   
39655095         2  2022-05-31 17:46:00   2022-05-31 17:51:00   
39655713         2  2022-05-31 19:02:47   2022-05-31 19:02:57   
39655730         2  2022-05-31 19:25:00   2022-05-31 19:25:09   
39655857         2  2022-05-31 21:18:44   2022-05-31 21:19:03   
39656093         2  2022-05-31 23:40:19   2022-06-01 00:01:20   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
293                   1.0            0.0         2.0                  N   
294        

### logical test 3 fare amount consistency 
I made sure that the fare adds up so that it all makes sense and than creating a column again to see if its true or not.

In [7]:
# Check for Fare Amount Consistency
fare_columns = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
df['inconsistent_fare_amount'] = abs(df['fare_amount'] - df[fare_columns].sum(axis=1)) < 0.01

# Print the records with inconsistent fare amounts, if any
inconsistent_fares = df[df['inconsistent_fare_amount']]
if  inconsistent_fares.empty:
    print("Inconsistent fare amounts detected in the following records:")
    print(inconsistent_fares)
else:
    print("All fare amounts are consistent.")

All fare amounts are consistent.


### Logical tests  4 make sure total amount is above 0. 
A journey would not be negative money so just to identify these rows and see if they are consistent with other rows identified and just having a column to say true or not.

In [8]:
# Check for Reasonable Total Amounts
df['negative_total_amount'] = df['total_amount'] < 0

# Print the records with negative total amounts, if any
negative_total_amounts = df[df['negative_total_amount']]
if not negative_total_amounts.empty:
    print("Negative total amounts detected in the following records:")
    print(negative_total_amounts)
else:
    print("All total amounts are reasonable.")

Negative total amounts detected in the following records:
          VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
86               2  2022-10-01 00:52:07   2022-10-01 01:02:44   
110              2  2022-10-01 00:29:57   2022-10-01 00:34:37   
162              2  2022-10-01 00:31:37   2022-10-01 00:32:53   
293              2  2022-10-01 00:46:38   2022-10-01 00:46:44   
319              2  2022-10-01 00:06:08   2022-10-01 00:48:23   
...            ...                  ...                   ...   
39638854         2  2022-05-26 15:17:09   2022-05-26 15:18:19   
39639116         2  2022-05-26 16:20:04   2022-05-26 17:29:38   
39650635         2  2022-05-30 01:18:00   2022-05-30 01:36:00   
39651466         2  2022-05-30 12:02:00   2022-05-30 12:47:00   
39654046         2  2022-05-31 10:35:00   2022-05-31 10:38:00   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
86                    1.0           2.83         1.0                  N   
110        

## All logical tests together combined
Ok I just wanted to see how many issues were so i got Next is to see if there is an overlap of it so could just be some trips arent recorded correctly and that all of these factors could identifty that.
Column Counts:
invalid_trip_distance       574059
inconsistent_fare_amount      9152
negative_total_amount       255706
dtype: int64
All columns have at least one True value: True
Number of records with all columns as True or partially True: 796029

In [9]:
# Count the number of True values in each column
column_counts = df[['invalid_trip_distance', 'inconsistent_fare_amount', 'negative_total_amount','invalid_time']].sum()

# Check if all columns have at least one True value
all_columns_have_true = all(column_counts > 0)

# Check the number of records that have all columns as True or partially True
all_columns_true_records = df[( df['invalid_time'] | df['invalid_trip_distance'] | df['inconsistent_fare_amount'] | df['negative_total_amount'])].shape[0]

# Print the results
print("Column Counts:")
print(column_counts)
print("All columns have at least one True value:", all_columns_have_true)
print("Number of records with all columns as True or partially True:", all_columns_true_records)

Column Counts:
invalid_trip_distance       574059
inconsistent_fare_amount      9152
negative_total_amount       255706
invalid_time                 13613
dtype: int64
All columns have at least one True value: True
Number of records with all columns as True or partially True: 809363


Ok below is just deleting all the rows that have True in all of the categories just to see if there is an overlap because in some there was. So to elimante those rows.

In [10]:
# Delete rows where all columns are True
df = df[~(df['invalid_time'] & df['invalid_trip_distance'] & df['inconsistent_fare_amount'] & df['negative_total_amount'])]

Error checking to see if there was any. In some of the tests I ran there was so Idk why this happened only once or twice or some of them would be like 10 or so rows. Another time I ran it, it was 9000 rows were deleted so yeah just for me.

In [11]:
column_counts_after_deletion = df[['invalid_time','invalid_trip_distance', 'inconsistent_fare_amount', 'negative_total_amount']].sum()
# Print the results
print("Column Counts after deletion:")
print(column_counts_after_deletion)

Column Counts after deletion:
invalid_time                 13613
invalid_trip_distance       574059
inconsistent_fare_amount      9152
negative_total_amount       255706
dtype: int64


Deleting all these rows because they make no sense if they are true so just deleting all those rows. As it runs the integrity of the data. That these could not be possible. Than below that checking how many were true. So i got 0 everytime I ran it so all were deleted. And than im going to drop those columns.

In [12]:
df = df[~df['invalid_trip_distance']]
df=df[~df['inconsistent_fare_amount']]
df=df[~df['negative_total_amount']]
df=df[~df['invalid_time']]

In [13]:
column_counts_now = df[['invalid_time','invalid_trip_distance', 'inconsistent_fare_amount', 'negative_total_amount']].sum()
# Print the results
print("Column Counts after deletion:")
print(column_counts_now)

Column Counts after deletion:
invalid_time                0
invalid_trip_distance       0
inconsistent_fare_amount    0
negative_total_amount       0
dtype: int64


In [14]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'invalid_time',
       'invalid_trip_distance', 'inconsistent_fare_amount',
       'negative_total_amount'],
      dtype='object')

## Ok below is logical test 5 to ensure the dates match and are not in the wrong order. 

As in no time traveling is occuring.

In [15]:
# Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' to datetime type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [16]:
# Split 'tpep_pickup_datetime' into date and time columns
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['pickup_time'] = df['tpep_pickup_datetime'].dt.time

# Split 'tpep_dropoff_datetime' into date and time columns
df['dropoff_date'] = df['tpep_dropoff_datetime'].dt.date
df['dropoff_time'] = df['tpep_dropoff_datetime'].dt.time

# Remove the original 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns
# df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

## Have to check this logic again. It sometimes works. Other times does not. 
# I got 100 rows in this in one of them and than another time i have 3773428 so have to check.

In [17]:
# Filter the DataFrame for rows where the pickup and dropoff dates do not match
mismatched_dates = df[df['pickup_date'] != df['dropoff_date']]
print("Number of rows with mismatched pickup and dropoff dates:", len(mismatched_dates))

Number of rows with mismatched pickup and dropoff dates: 377428


Below I just dropped the rows of logical test I did.

I just wanted to check i dropped them because I did not for awhile and was wondering what was wrong. I than just wanted to check how many rows i have left and columns.

In [18]:
# Drop columns 'inconsistent_fare_amount', 'negative_total_amount', and 'invalid_trip_distance'
dfcnew = df.drop(columns=['invalid_time','inconsistent_fare_amount', 'negative_total_amount', 'invalid_trip_distance'])

In [19]:
dfcnew.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'pickup_date',
       'pickup_time', 'dropoff_date', 'dropoff_time'],
      dtype='object')

In [20]:
dfcnew.shape

(38846735, 23)

I have all the Nans here just to see. None are important to our analysis.Mostly are passenger counts and ratcodeid and store fwd flag including congestion surcharge and airport fee which are not important in our analysis and see to be the same rows.

In [21]:
nan_counts = dfcnew.isna().sum()
print("NaN counts for each column:")
print(nan_counts)

NaN counts for each column:
VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1282655
trip_distance                  0
RatecodeID               1282655
store_and_fwd_flag       1282655
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1282655
airport_fee              1282655
pickup_date                    0
pickup_time                    0
dropoff_date                   0
dropoff_time                   0
dtype: int64


In [22]:
dfcnew.shape

(38846735, 23)

In [23]:
# Delete rows with NaN values
dfcnew.dropna(inplace=True)

In [24]:
dfcnew.shape

(37564080, 23)

In [25]:
nan_counts = dfcnew.isna().sum()
print("NaN counts for each column:")
print(nan_counts)

NaN counts for each column:
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
pickup_date              0
pickup_time              0
dropoff_date             0
dropoff_time             0
dtype: int64


I want to see how many duplicates there are and than delete them maybe

In [26]:
duplicates = dfcnew.duplicated()
print(duplicates.sum())

0


In [27]:
dfcnew.shape

(37564080, 23)

In [28]:
missing_values = duplicates.isnull().sum()
print(missing_values)

0


In [29]:
dfcnew.shape

(37564080, 23)

In [ ]:
# duplicate_rows = dfcnew.duplicated(keep=False)
# print(duplicate_rows)

In [31]:
dfcnew.shape

(37564080, 23)

In [32]:
cleaneddata = dfcnew.drop_duplicates()

In [33]:
cleaneddata.shape

(37564080, 23)

In [ ]:
print(cleaneddata.columns)
filtered_table = pa.Table.from_pandas(cleaneddata)
pq.write_table(filtered_table, 'cleaneddata.parquet')

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'pickup_date',
       'pickup_time', 'dropoff_date', 'dropoff_time'],
      dtype='object')


## All below are just for analysing the data. 
i broke them down into months, hours, and season, holidays. 

Ok firstly I want the month and I just want to order them because it wasnt ordered in mine. And change it to category this is just for analysis though.

In [ ]:
# Extract the month from the datetime columns
cleaneddata['pickup_month'] = cleaneddata['tpep_pickup_datetime'].dt.month_name()
cleaneddata['dropoff_month'] = cleaneddata['tpep_dropoff_datetime'].dt.month_name()

# Define the desired order of months
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Convert the 'pickup_month' and 'dropoff_month' columns to categorical type with the specified order
cleaneddata['pickup_month'] = pd.Categorical(cleaneddata['pickup_month'], categories=month_order, ordered=True)
cleaneddata['dropoff_month'] = pd.Categorical(cleaneddata['dropoff_month'], categories=month_order, ordered=True)

# Sort the DataFrame by the 'pickup_month' column
df = cleaneddata.sort_values('pickup_month')

This one is to seperate the time and date into two different columns. Just for easier analysis latewr on

I have this test to see if the trip was in one day or a few days like. I wanted to see if it was plausible .As in maybe the trip duration 

In [ ]:
def round_to_nearest_hour(time):
    rounded_time = time.replace(minute=0, second=0)
    return rounded_time

df['dropoff_time'] = df['dropoff_time'].apply(round_to_nearest_hour)
df['pickup_time'] = df['pickup_time'].apply(round_to_nearest_hour)

In [ ]:
# Identify holidays
cal = calendar()
holidays = cal.holidays(start='2022-01-01', end='2022-12-31')
# Convert 'pickup_date' to datetime-like object
df['pickup_date'] = pd.to_datetime(df['pickup_date'])

# Check if the pickup date is a holiday and assign day types
df['holiday'] = df['pickup_date'].isin(holidays)
df['week'] = df['pickup_date'].dt.dayofweek
df.loc[df['week'] >= 5, 'day_type'] = "weekend"
df.loc[df['week'] < 5, 'day_type'] = "workday"
df.loc[df['holiday'] == True, 'day_type'] = "holiday"

In [ ]:
print(df.head())

In [ ]:
# # Drop the 'holiday' and 'week' columns
# df = df.drop(['holiday', 'week'], axis=1)

# Define a function to assign time slots
def time_slots(x):
    if x.hour in range(6, 12):
        return 'Morning'
    elif x.hour in range(12, 17):
        return 'Afternoon'
    elif x.hour in range(17, 22):
        return 'Evening'
    else:
        return 'Late Night'

# Apply the time_slots function to 'tpep_pickup_datetime'
df['dropoff_timeslots'] = df['dropoff_time'].apply(time_slots)
df['pickup_timeslots'] = df['pickup_time'].apply(time_slots)

# Print the updated DataFrame
# print(df.head())

Ok below is seasons just to make sure I have all the seasons here so that I can analysis the data.

In [ ]:
df.dtypes

In [ ]:
winter_start = pd.to_datetime('2022-01-01').date()
winter_end = pd.to_datetime('2022-02-28').date()
spring_start = pd.to_datetime('2022-03-01').date()
spring_end = pd.to_datetime('2022-05-31').date()
summer_start = pd.to_datetime('2022-06-01').date()
summer_end = pd.to_datetime('2022-08-31').date()
autumn_start = pd.to_datetime('2022-09-01').date()
autumn_end = pd.to_datetime('2022-11-30').date()

# Set the 'season' value based on the date range
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
df['season'] = pd.NA
df.loc[(df['tpep_pickup_datetime'] >= winter_start) & (df['tpep_pickup_datetime'] <= winter_end), 'season'] = 'winter'
df.loc[(df['tpep_pickup_datetime'] >= spring_start) & (df['tpep_pickup_datetime'] <= spring_end), 'season'] = 'spring'
df.loc[(df['tpep_pickup_datetime'] >= summer_start) & (df['tpep_pickup_datetime'] <= summer_end), 'season'] = 'summer'
df.loc[(df['tpep_pickup_datetime'] >= autumn_start) & (df['tpep_pickup_datetime'] <= autumn_end), 'season'] = 'autumn'

# Filter the DataFrame for each season
winter_data = df[df['season'] == 'winter']
spring_data = df[df['season'] == 'spring']
summer_data = df[df['season'] == 'summer']
autumn_data = df[df['season'] == 'autumn']

# Count the number of NaN values in the 'season' column
nan_count = df['season'].isna().sum()

# Print the results
print("Number of NaN values in 'season' column:", nan_count)


In [ ]:
# Get the unique values for the "passenger" column
passenger_values = df['passenger_count'].unique()

# Print the unique values
print("Unique passenger values:")
print(passenger_values)

# Count the occurrences of each unique value in the "passenger" column
passenger_counts = df['passenger_count'].value_counts()

# Print the counts
print("Value counts for passenger:")
print(passenger_counts)

In [ ]:
passenger_counts = df['passenger_count'].value_counts()

# Calculate the percentage for each value
passenger_percentage = passenger_counts / len(df) * 100

# Print the percentages
print("Percentage for each passenger count:")
print(passenger_percentage)

In [ ]:
# convert passenger amount into three level
def passenger_convert(row):
    if (row >= 1) & (row <= 2):
        val = "small"
    elif (row > 2) & (row < 5):
        val = "medium"
    else:
        val = "high"
    return val

df['passenger'] = df['passenger_count'].apply(passenger_convert)
df['passenger'].value_counts()

In [ ]:
print(df.dtypes) 

In [ ]:
# Assuming 'combined_df' is your DataFrame and 'column1', 'column2', 'column3' are the columns you want to drop
combination = df.drop(['trip_distance','VendorID', 'tolls_amount', 'extra','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','airport_fee','store_and_fwd_flag','tip_amount','mta_tax','payment_type','fare_amount','RatecodeID'], axis=1)

In [ ]:
print(combination)

In [ ]:
# Drop rows with any missing values
combinations = combination.dropna()

In [ ]:
print(combinations.columns)
filtered_table = pa.Table.from_pandas(combinations)
pq.write_table(filtered_table, 'dataforanalysing.parquet')